In [2]:
import pandas as pd
import seaborn as sns
import pathlib as Path
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import os
print(os.listdir("../input"))

['train.csv', 'sample_submission.csv', 'test.csv']


In [9]:
df = pd.read_csv("../input/train.csv")
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df = df[df['trip_duration'] <= 5000]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1454871 entries, 0 to 1458643
Data columns (total 11 columns):
id                    1454871 non-null object
vendor_id             1454871 non-null int64
pickup_datetime       1454871 non-null datetime64[ns]
dropoff_datetime      1454871 non-null object
passenger_count       1454871 non-null int64
pickup_longitude      1454871 non-null float64
pickup_latitude       1454871 non-null float64
dropoff_longitude     1454871 non-null float64
dropoff_latitude      1454871 non-null float64
store_and_fwd_flag    1454871 non-null object
trip_duration         1454871 non-null int64
dtypes: datetime64[ns](1), float64(4), int64(3), object(3)
memory usage: 133.2+ MB


In [10]:
## split pickup_datetime
df['pickup_hour'] = df['pickup_datetime'].dt.hour
df['pickup_minute'] = df['pickup_datetime'].dt.minute
df['pickup_second'] = df['pickup_datetime'].dt.second

In [11]:
df.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_hour,pickup_minute,pickup_second
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455,17,24,55
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663,0,43,35
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124,11,35,24
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429,19,32,31
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435,13,30,55


In [12]:
## X creation
X = df[[
    'pickup_longitude', 
    'pickup_latitude', 
    'dropoff_longitude', 
    'dropoff_latitude', 
    'pickup_hour',
    'pickup_minute',
    'pickup_second',
   ]]

In [13]:
## Y creation
Y = df[['trip_duration']]

## Shape
X.shape, Y.shape

((1454871, 7), (1454871, 1))

In [14]:
## Cross validation
rf = RandomForestRegressor()
score = -cross_val_score(rf, X, Y, cv=5, scoring='neg_mean_squared_log_error')
score.mean()

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:528: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_trai

0.17934298397973417

In [15]:
rf.fit(X, Y)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [16]:
## DF Test creation
df_test = pd.read_csv('../input/test.csv')
df_test['pickup_datetime'] = pd.to_datetime(df_test['pickup_datetime'])
df_test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,N
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,N
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,N
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,N
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,N


In [17]:
## split pickup_datetime
df_test['pickup_hour'] = df_test['pickup_datetime'].dt.hour
df_test['pickup_minute'] = df_test['pickup_datetime'].dt.minute
df_test['pickup_second'] = df_test['pickup_datetime'].dt.second

In [20]:
## X creation
X_test = df_test[[
    'pickup_longitude', 
    'pickup_latitude', 
    'dropoff_longitude', 
    'dropoff_latitude', 
    'pickup_hour',
    'pickup_minute',
    'pickup_second',
    ]]

In [21]:
## y prediction and mean for this prediction
y_pred = rf.predict(X_test)
y_pred.mean()

842.9742496808684

In [22]:
## submit creation
submission = pd.read_csv('../input/sample_submission.csv') 
submission.head()

,id,trip_duration
0,id3004672,959
1,id3505355,959
2,id1217141,959
3,id2150126,959
4,id1598245,959


In [23]:
## submission for the y prediction
submission['trip_duration'] = y_pred
submission.head()

,id,trip_duration
0,id3004672,731.6
1,id3505355,774.1
2,id1217141,566.1
3,id2150126,1232.2
4,id1598245,246.7


In [24]:
## describe submission
submission.describe()

,trip_duration
count,625134.000000
mean,842.974250
std,577.899593
min,5.900000
25%,441.700000
50%,690.450000
75%,1068.400000
max,4509.700000


In [25]:
## send submission
submission.to_csv('submission.csv', index=False)

In [26]:
!ls


__notebook_source__.ipynb  submission.csv
